<a href="https://colab.research.google.com/github/ariel07-sw/applied-data-in-econ/blob/main/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("WagePanel.xlsx")

In [6]:
nrs = [120, 126, 150]
df = df.rename(columns={'  1. nr                       person identifier': '1. nr'})
df = df[df["1. nr"].isin(nrs)].copy()

In [10]:
df = df.rename(columns={
    ' 19. lwage                    log(wage)': 'lwage',
    '  4. exper                    labor market experience': 'experience',
    '  7. married                  =1 if married': 'marriage'
})

if "year" in df.columns:
    df = df.sort_values(["1. nr","year"])
else:
    df = df.sort_values(["1. nr"])

y = df["lwage"]
x_exp = df["experience"]
x_mar = df["marriage"]

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
print(df.head())
print("N =", len(df))

    1. nr    2. year                     1980 to 1987  \
40  120.0                                      1980.0   
41  120.0                                      1981.0   
42  120.0                                      1982.0   
43  120.0                                      1983.0   
44  120.0                                      1984.0   

      3. black                    =1 if black  experience  \
40                                        0.0         2.0   
41                                        0.0         3.0   
42                                        0.0         4.0   
43                                        0.0         5.0   
44                                        0.0         6.0   

      5. hisp                     =1 if Hispanic  \
40                                           0.0   
41                                           0.0   
42                                           0.0   
43                                           0.0   
44                            

In [11]:
print(df["1. nr"].value_counts())
print("Unique IDs:", df["1. nr"].unique())
print("N =", len(df))

1. nr
120.0    8
126.0    8
150.0    8
Name: count, dtype: int64
Unique IDs: [120. 126. 150.]
N = 24


In [13]:
import statsmodels.api as sm
import pandas as pd

dummies = pd.get_dummies(df["1. nr"], drop_first=True)
dummies = dummies.astype(int) # Convert boolean dummies to integers

X = pd.concat([df[["experience", "marriage"]], dummies], axis=1)

X = sm.add_constant(X)

y = df["lwage"]

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.652
Model:                            OLS   Adj. R-squared:                  0.579
Method:                 Least Squares   F-statistic:                     8.899
Date:                Tue, 24 Feb 2026   Prob (F-statistic):           0.000318
Time:                        03:26:36   Log-Likelihood:                -5.1418
No. Observations:                  24   AIC:                             20.28
Df Residuals:                      19   BIC:                             26.17
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0414      0.224      4.659      0.0

In [14]:

df_demeaned = df.copy()

for var in ["lwage", "experience", "marriage"]:
    df_demeaned[var] = df[var] - df.groupby("1. nr")[var].transform("mean")

X_dm = df_demeaned[["experience", "marriage"]]
y_dm = df_demeaned["lwage"]

model_dm = sm.OLS(y_dm, X_dm).fit()

print(model_dm.summary())

                                 OLS Regression Results                                
Dep. Variable:                  lwage   R-squared (uncentered):                   0.196
Model:                            OLS   Adj. R-squared (uncentered):              0.123
Method:                 Least Squares   F-statistic:                              2.689
Date:                Tue, 24 Feb 2026   Prob (F-statistic):                      0.0902
Time:                        03:28:01   Log-Likelihood:                         -5.1418
No. Observations:                  24   AIC:                                      14.28
Df Residuals:                      22   BIC:                                      16.64
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [16]:
!pip -q install linearmodels

from linearmodels.panel import PanelOLS
import pandas as pd

df = df.copy()
df = df.rename(columns={'  2. year                     1980 to 1987': 'year'})
df["year"] = df["year"].astype(int)

df_panel = df.set_index(["1. nr", "year"])

Y = df_panel["lwage"]
X = df_panel[["experience", "marriage"]]

fe_res = PanelOLS(Y, X, entity_effects=True).fit()

print(fe_res.summary)
print("\nFE coefficients:\n", fe_res.params)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1965
Estimator:                   PanelOLS   R-squared (Between):              0.4034
No. Observations:                  24   R-squared (Within):               0.1965
Date:                Tue, Feb 24 2026   R-squared (Overall):              0.3942
Time:                        03:39:53   Log-likelihood                   -5.1418
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2.3227
Entities:                           3   P-value                           0.1252
Avg Obs:                       8.0000   Distribution:                    F(2,19)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             2.3227
                            

In [18]:
import pandas as pd

df = pd.read_excel("murder.xlsx")
df.head()

,stateID,State,year,mrdrte,exec,unem,d93,d90,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1,"""AL""",87,9.3,2,7.8,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"""AK""",87,10.1,0,10.8,0,0,NaN,NaN,Obs:,153,NaN,NaN,NaN,NaN,NaN
2,3,"""AZ""",87,7.5,0,6.2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,"""AR""",87,7.6,0,8.1,0,0,NaN,NaN,1,id,state,identifier,NaN,NaN,NaN
4,5,"""CA""",87,10.6,0,5.8,0,0,NaN,NaN,2,state,postal,code,NaN,NaN,NaN


In [24]:

df_9193 = df[df["year"].isin([91,92,93])]

exec_states = df_9193.groupby("State")["exec"].sum()

num_states = (exec_states > 0).sum()

print("States with at least one execution:", num_states)


States with at least one execution: 16


In [25]:
most_exec = exec_states.idxmax()
max_value = exec_states.max()

print("State with most executions:", most_exec)
print("Number of executions:", max_value)

State with most executions: "TX"
Number of executions: 34


In [26]:
df["d93"] = (df["year"] == 1993).astype(int)

df_90_93 = df[df["year"].isin([1990,1993])]

In [28]:
import statsmodels.api as sm

df["d93"] = (df["year"] == 93).astype(int)
df_90_93 = df[df["year"].isin([90,93])]

X = df_90_93[["d93","exec","unem"]]
X = sm.add_constant(X)
y = df_90_93["mrdrte"]

model = sm.OLS(y,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 mrdrte   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     3.695
Date:                Tue, 24 Feb 2026   Prob (F-statistic):             0.0144
Time:                        04:04:24   Log-Likelihood:                -379.81
No. Observations:                 102   AIC:                             767.6
Df Residuals:                      98   BIC:                             778.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.2780      4.428     -1.192      0.2

In [30]:
df = df.sort_values(["State","year"])

In [33]:
df_diff = df[df["year"].isin([90,93])]

df_diff = df_diff.pivot(index="State", columns="year")

delta_mrdrte = df_diff["mrdrte"][93] - df_diff["mrdrte"][90]
delta_exec   = df_diff["exec"][93] - df_diff["exec"][90]
delta_unem   = df_diff["unem"][93] - df_diff["unem"][90]

df_final = pd.DataFrame({
    "delta_mrdrte": delta_mrdrte,
    "delta_exec": delta_exec,
    "delta_unem": delta_unem
}).dropna()

print("Observations:", len(df_final))

Observations: 51


In [34]:
X = df_final[["delta_exec","delta_unem"]]
X = sm.add_constant(X)
y = df_final["delta_mrdrte"]

model_diff = sm.OLS(y,X).fit()
print(model_diff.summary())

                            OLS Regression Results                            
Dep. Variable:           delta_mrdrte   R-squared:                       0.110
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     2.959
Date:                Tue, 24 Feb 2026   Prob (F-statistic):             0.0614
Time:                        04:12:25   Log-Likelihood:                -74.693
No. Observations:                  51   AIC:                             155.4
Df Residuals:                      48   BIC:                             161.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4133      0.209      1.974      0.0